# Analisi

In [1]:
import geopandas as gdp

from my_paths import *

In [17]:
gdf_ciclabili = gdp.read_file(PATH_CICLABILI_CLEAN).to_crs("EPSG:32632")
gdf_bikemi = gdp.read_file(PATH_BIKEMI_CLEAN).to_crs("EPSG:32632")
gdf_parchi = gdp.read_file(PATH_PARCHI_CLEAN).to_crs("EPSG:32632")
gdf_biblioteche = gdp.read_file(PATH_BIBLIOTECHE_CLEAN).to_crs("EPSG:32632")
gdf_impianti_sportivi = gdp.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN).to_crs("EPSG:32632")
gdf_fontane = gdp.read_file(PATH_FONTANE_CLEAN).to_crs("EPSG:32632")
gdf_inquinamento = gdp.read_file(PATH_INQUINAMENTO_INGESTION_CLEAN).to_crs("EPSG:32632")

In [ ]:
buffer_union = gdf_ciclabili.buffer(1000).unary_union
parchi_non_vicini = gdf_parchi[~gdf_parchi.geometry.intersects(buffer_union)]

In [ ]:
#print(gdf_parchi.info())
print(parchi_non_vicini)